# OPIM510: Week 3 - Window Functions

## Window Functions

Defined by [Postgres](https://www.postgresql.org/docs/9.1/tutorial-window.html) as:

> A *window function* performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row --- the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

### Examples of Window Function use cases

- Ranking (or ranking within groups) based on a metric
- Running totals (or running totals within groups) of a metric
- Lead/lag calculations (i.e. Looking forward or backwards by *n* rows to perform a calculation on the current row)

## Prepare our environment

In [ ]:
import polars as pl
from datetime import datetime, date

# Configure Polars for better display
pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(50)

## Load our dataset

In [ ]:
# Read the Superstore Excel file
sales = pl.read_csv('data/Superstor.csv')
print(f"Shape: {sales.shape}")
print(f"\nColumns: {sales.columns}")
sales.head()

## Using window functions to compute ranking

**Business Question:** Who are our top 10 most `Profit`able `Customer Name`s?

Key Points:
- We need to aggregate `Profit` for each `Customer Name`
- We need to rank the `Customer Name` groups on total `Profit`, then filter for only the top 10

In [ ]:
top_customers = (
    sales
    .group_by("Customer Name")
    .agg(
        pl.col("Profit").sum().alias("total_profit")
    )
    .with_columns(
        pl.col("total_profit").rank(method="min", descending=True).alias("profit_rank")
    )
    .filter(pl.col("profit_rank") < 11)
    .sort("profit_rank")
)

top_customers

## Using window function within groups

**Business Question:** Who are the most `Profit`able `Customer Names` for each `Region`?

Key Points:
- We need to aggregate `Profit` for each combination of `Customer Name` and `Region`
- We need to rank the `Customer Name` groups on total `Profit`, then filter for only the top 10 FOR EACH `Region`

In [ ]:
top_customers_region = (
    sales
    .group_by(["Customer Name", "Region"])
    .agg(
        pl.col("Profit").sum().alias("total_profit")
    )
    .with_columns(
        pl.col("total_profit")
        .rank(method="min", descending=True)
        .over("Region")
        .alias("profit_rank")
    )
    .filter(pl.col("profit_rank") < 11)
    .sort(["Region", "profit_rank"])
)

print(f"Shape: {top_customers_region.shape}")
top_customers_region.head(20)

## Cumulative (running) totals

**Business Question:** Show the running total of `Sales` for the days (using by `Order Date`) leading up to 2016-11-08.

Key Points:
- Need to figure out how to get a running total of `Sales`
- Must order the data in ascending order by `Order Date`

In [ ]:
running_sales = (
    sales
    .group_by("Order Date")
    .agg(
        pl.col("Sales").sum().alias("tot_sales")
    )
    .sort("Order Date")
    .with_columns(
        pl.col("tot_sales").cum_sum().alias("running_total")
    )
    .filter(pl.col("Order Date") < datetime(2016, 11, 8))
)

print(f"Shape: {running_sales.shape}")
running_sales.tail(10)

## Quick Summarization

**Business Question:** What is the the total amount of both `Sales` and `Profit` for each month of `Order Date`? (When computing months, each combination of year and month counts as a unique month.)

Key points:
- Need to figure out how to parse out only the year/months from the `Order Date` column
- We can use Polars' datetime functionality to truncate dates to month boundaries
- FOR EACH usually means we are grouping by something. In this case we have to figure out how to get year/month combinations from the `Order Date` field
- Create sums for both `Sales` and `Profit`

### Date truncation examples in Polars

In [ ]:
# Examples of date truncation in Polars
example_date = datetime(2023, 11, 7, 12, 31, 13)
df_example = pl.DataFrame({"datetime": [example_date]})

print("Original datetime:", example_date)
print("\nTruncated to different units:")
print(df_example.with_columns([
    pl.col("datetime").dt.truncate("1y").alias("year"),
    pl.col("datetime").dt.truncate("1mo").alias("month"),
    pl.col("datetime").dt.truncate("1d").alias("day"),
    pl.col("datetime").dt.truncate("1h").alias("hour"),
    pl.col("datetime").dt.truncate("1m").alias("minute"),
]))

In [ ]:
monthly_sales = (
    sales
    .with_columns(
        pl.col("Order Date").dt.truncate("1mo").alias("month")
    )
    .group_by("month")
    .agg([
        pl.col("Sales").sum().alias("tot_sales"),
        pl.col("Profit").sum().alias("tot_profit")
    ])
    .sort("month")
)

print(f"Shape: {monthly_sales.shape}")
monthly_sales.head(12)

## Going one level further

**Business Question:** Using the data frame from the previous question: between which months was the greatest jump in sales?

Key Points:
- Using the previous data frame
- Need to figure out the difference between `tot_sales` from month to month

In [ ]:
delta_sales = (
    monthly_sales
    .with_columns(
        (pl.col("tot_sales") - pl.col("tot_sales").shift(1)).alias("month_sales_delta")
    )
)

print("\nMonth with greatest sales increase:")
print(delta_sales.sort("month_sales_delta", descending=True).head(1))

print("\nAll monthly deltas:")
delta_sales.head(12)

## Same dataset but now finding the `Profit` monthly delta

**Business Question:** Using the data frame from the previous question: between which months was the greatest jump in profit?

Key Points:
- Using the previous data frame
- Need to figure out the difference between `tot_profit` from month to month

In [ ]:
delta_sales = (
    delta_sales
    .with_columns(
        (pl.col("tot_profit") - pl.col("tot_profit").shift(1)).alias("month_profit_delta")
    )
)

print("\nMonth with greatest profit increase:")
print(delta_sales.sort("month_profit_delta", descending=True).head(1))

print("\nAll monthly deltas:")
delta_sales.head(12)

## Additional Window Function Examples

### Lead function example
Let's look at the next month's sales alongside current month

In [ ]:
# Using shift with negative value for lead
lead_example = (
    monthly_sales
    .with_columns([
        pl.col("tot_sales").shift(-1).alias("next_month_sales"),
        pl.col("tot_sales").shift(1).alias("prev_month_sales")
    ])
)

print("Lead/Lag example:")
lead_example.head(10)

### Rolling window calculations
Calculate 3-month moving average of sales

In [ ]:
rolling_example = (
    monthly_sales
    .sort("month")
    .with_columns(
        pl.col("tot_sales").rolling_mean(window_size=3).alias("3_month_avg_sales")
    )
)

print("Rolling 3-month average:")
rolling_example.head(12)